In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import _pickle as pickle
from os import listdir
from os.path import join, isfile

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
from keras import backend as K
K.set_session(session)

from keras.preprocessing import image
from keras.utils import to_categorical, plot_model

from keras.applications import xception
from keras.applications import inception_v3

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score


Using TensorFlow backend.


In [2]:
def read_img(img_id, train_or_test, size=None):
    img = image.load_img(join(data_dir, train_or_test, '%s.jpg' % img_id), target_size=size)
    img = image.img_to_array(img)
    return img

In [3]:
# Load labels and split to train/val
NUM_CLASSES = 120
SEED = 1993
np.random.seed(seed=SEED)
data_dir = '../data'
labels = pd.read_csv(join(data_dir, 'labels.csv'))
l_val = labels.groupby('breed').apply(pd.DataFrame.sample, frac=0.2).reset_index(drop=True)
l_tr = labels.loc[~labels['id'].isin(l_val['id'])]
sample_submission = pd.read_csv(join(data_dir, 'sample_submission.csv'))
print('Train:', len(listdir(join(data_dir, 'train'))), '=', len(l_tr)+len(l_val), '=', len(l_val), '+', len(l_tr))
print('Test:', len(listdir(join(data_dir, 'test'))), '=', len(sample_submission))

print("We should have {} clasess and the train data has {} classes."
      .format(NUM_CLASSES, len(l_tr.groupby('breed').count())))

l_tr_index = {label:i for i,label in enumerate(np.unique(l_tr.breed))}
l_tr_temp = [l_tr_index[label] for label in l_tr.breed]
l_val_temp = [l_tr_index[label] for label in l_val.breed]
y_tr = to_categorical(l_tr_temp ,num_classes=120)
y_val = to_categorical(l_val_temp ,num_classes=120)

print('y_tr shape: {}'.format(y_tr.shape))
print('y_val shape: {}'.format(y_val.shape))

Train: 10225 = 10222 = 2037 + 8185
Test: 10359 = 10357
We should have 120 clasess and the train data has 120 classes.
y_tr shape: (8185, 120)
y_val shape: (2037, 120)


In [6]:
# Get bottelneck features
INPUT_SIZE = 299
POOLING = 'avg'
filename = data_dir + '//train//xs_bf_inception_v3'

if isfile(filename):
    # Load from file
    print('Loading from {}'.format(filename))
    with open(filename, 'rb') as fp:
        xs_bf = pickle.load(fp)
    print('xs_bf shape: {} size: {:,}'.format(xs_bf.shape, xs_bf.size))
else:
    # Preprocess images
    print('Reading from raw images')
    xs = np.zeros((len(labels), INPUT_SIZE, INPUT_SIZE, 3), dtype='float32')
    for i, img_id in enumerate(labels['id']):
        img = read_img(img_id, 'train', (INPUT_SIZE, INPUT_SIZE))
        x = xception.preprocess_input(np.expand_dims(img.copy(), axis=0))
        xs[i] = x
    print('All train Images shape: {} size: {:,}'.format(xs.shape, xs.size))
    
    # Predict
    preprocess_bottleneck = inception_v3.InceptionV3(weights='imagenet', include_top=False, pooling=POOLING)
    xs_bf_raw = preprocess_bottleneck.predict(xs, batch_size=6, verbose=1)
    print('All train bottleneck features shape: {} size: {:,}'.format(xs_bf_raw.shape, xs_bf_raw.size))
    xs_bf = pd.concat([labels.id, pd.DataFrame(xs_bf_raw)], axis=1)
    
    # Save into file
    print('Dumping into {}'.format(filename))
    with open(filename, 'wb') as fp:
        pickle.dump(xs_bf, fp)

# Split to train/val sets
x_tr = np.zeros((len(l_tr), xs_bf.shape[1]-1), dtype='float32')
for i, img_id in enumerate(l_tr['id']):
    x_tr[i] = xs_bf[xs_bf.id == img_id].values[0][1:]
print('Train bottleneck features shape: {} size: {:,}'.format(x_tr.shape, x_tr.size))

x_val = np.zeros((len(l_val), xs_bf.shape[1]-1), dtype='float32')
for i, img_id in enumerate(l_val['id']):
    x_val[i] = xs_bf[xs_bf.id == img_id].values[0][1:]
print('Validation bottleneck features shape: {} size: {:,}'.format(x_val.shape, x_val.size))

xs = None
xs_bf = None
xs_bf_raw = None

Loading from ../data//train//xs_bf_inception_v3
xs_bf shape: (10222, 2049) size: 20,944,878
Train bottleneck features shape: (8185, 2048) size: 16,762,880
Validation bottleneck features shape: (2037, 2048) size: 4,171,776


In [8]:
# Logistic regression on bottleneck features
logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=SEED)
logreg.fit(x_tr, (y_tr * range(NUM_CLASSES)).sum(axis=1))
valid_probs = logreg.predict_proba(x_val)
valid_preds = logreg.predict(x_val)
print('Validation LogLoss {}'.format(log_loss(y_val, valid_probs)))
print('Validation Accuracy {}'.format(accuracy_score((y_val * range(NUM_CLASSES)).sum(axis=1), valid_preds)))

Validation LogLoss 0.3684238395832664
Validation Accuracy 0.8880706921944035


In [4]:
# Get test bottelneck features
INPUT_SIZE = 299
POOLING = 'avg'
filename = data_dir + '//test//xs_bf_inception'

if isfile(filename):
    print('File {} already exists!'.format(filename))
else:
    # Preprocess images
    print('Reading from raw images')
    xs = np.zeros((len(sample_submission.id), INPUT_SIZE, INPUT_SIZE, 3), dtype='float32')
    for i, img_id in enumerate(sample_submission['id']):
        img = read_img(img_id, 'test', (INPUT_SIZE, INPUT_SIZE))
        x = inception_v3.preprocess_input(np.expand_dims(img.copy(), axis=0))
        xs[i] = x
    print('All test Images shape: {} size: {:,}'.format(xs.shape, xs.size))
    
    # Predict
    preprocess_bottleneck = inception_v3.InceptionV3(weights='imagenet', include_top=False, pooling=POOLING)
    xs_bf_raw = preprocess_bottleneck.predict(xs, batch_size=6, verbose=1)
    print('All test bottleneck features shape: {} size: {:,}'.format(xs_bf_raw.shape, xs_bf_raw.size))
    xs_bf = pd.concat([sample_submission.id, pd.DataFrame(xs_bf_raw)], axis=1)
    
    # Save into file
    print('Dumping into {}'.format(filename))
    with open(filename, 'wb') as fp:
        pickle.dump(xs_bf, fp)

xs = None
xs_bf = None
xs_bf_raw = None

Reading from raw images
All test Images shape: (10357, 299, 299, 3) size: 2,777,778,471
10357/10357 [==============================] - 1000s  
All test bottleneck features shape: (10357, 2048) size: 21,211,136
Dumping into ../data//test//xs_bf_inception
